In [ ]:
#!unzip "drive/MyDrive/dog-breed-identification.zip" -d "drive/MyDrive/Dog Vision/"

# End-to-End Multi Class DOG BREED CLASSIFICATION

This notebook builds an End-to-end Multi-class Image classifier using TensorFlow 2.0 and TensorFlow Hub.

##1.Problem

Identifying the breed of a dog given an image of a dog.

##2. Data

The Data we're using is from Kaggle's Dog Breed Identification Competition.

https://www.kaggle.com/c/dog-breed-identification/data


##3. Evaluation

The Evaluation is a File with Prediction Probabilities for each dog breed of each test image.

##4. Features

Some information about the data:
* We're dealing with images (Unstructured Data) so its best we use DEEP LEARNING
* There are 120 breeds of Dogs (There are 120 Different Classes).
* There are around 10,000+ images in both Training and Test Sets.

# Get our Workspace Ready

* Import TensorFlow 2.x
* Import TensorFlow Hub
* Make sure we're using a GPU

In [ ]:
# Import TensorFlow into Colab
import tensorflow as tf
import tensorflow_hub as hub
print("TF Version:", tf.__version__)
print("TF Hub Version:", hub.__version__)

# Check for GPU Availability
print("GPU", "available" if tf.config.list_physical_devices("GPU") else "not available")

TF Version: 2.17.0
TF Hub Version: 0.16.1
GPU not available


## Getting our Data Ready (Turning into Tensors)

We Turn our images to Tensors(numerical representations).

Let's start by accessing our data and checking out the labels.


In [ ]:
import pandas as pd
labels_csv = pd.read_csv("/content/drive/MyDrive/Dog Vision/labels.csv")
labels_csv.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Dog Vision/labels.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
labels_csv.describe()

In [ ]:
# How many images are there of each breed

labels_csv["breed"].value_counts()

In [ ]:
labels_csv['breed'].value_counts().plot.bar(figsize=(20,10))

In [ ]:
labels_csv['breed'].value_counts().median()

In [ ]:
# Let's View an Image
from IPython.display import Image

Image("drive/MyDrive/Dog Vision/train/000bec180eb18c7604dcecc8fe0dba07.jpg")

### Getting Images and Their Labels

Let's get a list of all of our image file pathnames.

In [ ]:
labels_csv.head()

In [ ]:
filenames = ["drive/MyDrive/Dog Vision/train/" + fname for fname in labels_csv["id"].values + ".jpg"]
filenames[:10]

In [ ]:
# Check whether number of filenames matches number of actual image files
import os
if len(os.listdir("/content/drive/MyDrive/Dog Vision/train")) == len(filenames):
  print("Filenames match actual amount of files!")

In [ ]:
# One More Check
Image(filenames[9000])

In [ ]:
labels_csv["breed"][9000]

Since we now got our Training Image Filepaths in a list,
Let's prepare our labels.



In [ ]:
labels = labels_csv['breed']
labels

In [ ]:
import numpy as np
labels = np.array(labels)
labels

In [ ]:
# See if No. of labels matches the number of filenames
if len(labels) == len(filenames):
  print("Number of labels matches number of filenames!")

In [ ]:
unique_breeds = np.unique(labels)
len(unique_breeds)

In [ ]:
# Turn a Single Label into a Array of Booleans
print(labels[0])
labels[0] == unique_breeds

In [ ]:
# Turn every label into a Boolean Array
boolean_labels = [label == unique_breeds for label in labels]
len(boolean_labels)

In [ ]:
# Example: Turning boolean array into integers

print(labels[0])
print(np.where(unique_breeds == labels[0])[0][0]) # Index where Label Occurs
print(boolean_labels[0].argmax()) # Index where Label Occurs
print(boolean_labels[0].astype(int))

In [ ]:
print(labels[2])
print(boolean_labels[2].astype(int))

In [ ]:
filenames[:10]

### Creating our Own Validation Set

Since the dataset from Kaggle doesn't come with a Validation Set, we're going to create our own.

In [ ]:
# Setup X & y Variables

X = filenames
y = boolean_labels

In [ ]:
len(filenames)

We're going to start off Experimenting with ~1000 images and increase as needed.

In [ ]:
# Set Number of Images to use for Experimenting
NUM_IMAGES = 1000 #@param {type:"slider",min:1000,max:10000,step:1000}

In [ ]:
from sklearn.model_selection import train_test_split

np.random.seed(42)
# Split them into Training and Validation of Total size NUM_IMAGES
X_train, X_val, y_train, y_val = train_test_split(X[:NUM_IMAGES],
                                                  y[:NUM_IMAGES],
                                                  test_size=0.2)

len(X_train), len(y_train), len(X_val), len(y_val)

In [ ]:
X_train[:5], y_train[:2]

## Preprocessing Images (Turning Images into Tensors)

To Preprocess our images to Tensors, we're going to write a Function which does a Few things
1. Take an Image Filepath as Input
2. Use Tensorflow to read the file and save it to a variable `image`
3. Turn our `image` into TENSORS
4. Resize the image to be shape of (224,224)
5. Return the Modified Image

Before we do, let's see what importing an Image looks like.

In [ ]:
from matplotlib.pyplot import imread
image = imread(filenames[42])
image.shape

In [ ]:
image

In [ ]:
image.max() , image.min()

In [ ]:
# Turn image into a Tensor
tf.constant(image)

1.Take an Image Filepath as Input

2.Use Tensorflow to read the file and save it to a variable image

3.Turn our image into TENSORS

4. Normalise our Image (Convert Colour Channels values from 0-255 to 0-1)

4.Resize the image to be shape of (224,224)

5.Return the Modified Image

In [ ]:
IMG_SIZE = 224

# Create a Function for PreProcessing Images
def process_image(image_path,img_size=IMG_SIZE):
  """
  Takes an Image File Path and turns the image into a Tensor
  """
  # Read in an Image File
  image = tf.io.read_file(image_path)
  # Turn the JPEG Image to Numerical Tensor with 3 Colour Channels (R,G,B)
  image = tf.image.decode_jpeg(image, channels=3)
  # Convert the Colour Channel values from 0-255 to 0-1 values
  image = tf.image.convert_image_dtype(image,tf.float32)
  # Resize the Image
  image = tf.image.resize(image,size=[IMG_SIZE,IMG_SIZE])

  return image

## TURNING DATA INTO BATCHES

Why turn our Data into batches?

Let's say you're trying to process 10000 images in one go...they all might not fit into memory.

So that's why we do 32 images (BATCH SIZE) at a time.

In order to use TensorFlow effectively, we need our data in the form of Tensor Tuples which look like this: `(image,label)`

In [ ]:
# Create a Simple Function to return a Tuple (image, label)
def get_image_label(image_path, label):

  image = process_image(image_path)
  return image, label


In [ ]:
(process_image(X[42]), tf.constant(y[42]))

Now we've got a way to turn our data into tuples of Tensors in the form: `(image,label)` ,let's make a funciton to convert all of our data (X & y) into Batches.

In [ ]:
# Define the Batch Size, 32 is a Good Start

BATCH_SIZE = 32

# Create a Function to convert Data into Batches
def create_data_batches(X,y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
  """
  Creates batches of data out of image X and label y pairs.
  Shuffles the data if it's training data but doesn't shuffle if it's Validation Data.
  Also accepts test data as input (no labels).
  """
  # If the data is a Test Data, we probably don't have labels.

  if test_data:
    print("Creating Test Data Batches..")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X))) # Only Filepaths no Labels
    data_batch = data.map(process_image).batch(BATCH_SIZE)
    return data_batch

  elif valid_data:
    print('Creating Validation Data Batches..')
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X),
                                               tf.constant(y)))
    data_batch = data.map(get_image_label).batch(BATCH_SIZE)
    return data_batch
  else:
    print("Creating Training Data Batches...")
    # Turn FilePaths and Labels into Tensors
    data = tf.data.Dataset.from_tensor_slices((tf.constant(X),
                                               tf.constant(y)))

    # Shuffling Pathnames and Labels before mapping image processor function is faster than shuffling images
    data = data.shuffle(buffer_size=len(X))
    # Create (image,label) tuples .
    data = data.map(get_image_label)

    # Turn the Training Data into Batches
    data_batch = data.batch(BATCH_SIZE)

  return data_batch

In [ ]:
# Create Training and Validation Batches

train_data = create_data_batches(X_train, y_train)
val_data = create_data_batches(X_val, y_val, valid_data=True)

In [ ]:
train_data.element_spec, val_data.element_spec

### Visualising Data Batches

Our Data is now in batches, however, these can be a little hard to understand.
Let's Visualise them.

In [ ]:
import matplotlib.pyplot as plt

# Create a Function for Viewing Images in a Data Batch...
def show_25_images(images,labels):
  """
  Displays a Plot of 25 images and their labels from a data batch.
  """

  plt.figure(figsize=(10,10))
  for i in range(25):
    # Create Subplots (5 rows , 5 columns)
    ax = plt.subplot(5,5,i+1)
    plt.imshow(images[i])
    plt.title(unique_breeds[labels[i].argmax()])
    plt.axis("off")

In [ ]:
train_data

In [ ]:
# Now let's Visualise the Data in a Training Batch
train_images, train_labels = next(train_data.as_numpy_iterator())

show_25_images(train_images, train_labels)

In [ ]:
# Now let's visualise our Validation Set

val_images, val_labels = next(val_data.as_numpy_iterator())
show_25_images(val_images, val_labels)

## Building a Model

Before we build a Model, there are a few things we need to define

* The `input` shape (Our Images shape, in the form of Tensors) to our Model.

* The output Shape( Image Labels, in the form of Tensors) of our Model.

* The URL of the model we want to use.

In [ ]:
# Setup input shape to the Model

INPUT_SHAPE = [None, IMG_SIZE, IMG_SIZE, 3] # Batch, Height , Width , Color Channels

# Setup Output shape of our Model
OUTPUT_SHAPE = len(unique_breeds)

# Setup Model URL from Tensorflow Hub
MODEL_URL = "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4"


Now we've got our inputs , outputs and model ready to go.
Let's put them together into a Keras Deep Learning Model!

Knowing this, let's create a function which:
* Takes the input shape, output shape and the model we've chosen as parameters.
* Defines the Layers in a Keras Model in Sequential fashion.
* Compiles the Model (says how it should be evaluated and improved).
* Builds the model
* Returns the Model.

In [ ]:
!pip install tf-keras==2.15.1

In [ ]:
# Create a function which builds a Keras model
def create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, model_url=MODEL_URL):
  print("Building model with:", MODEL_URL)

  # Setup the model layers
  model = tf.keras.Sequential([
    hub.KerasLayer(MODEL_URL), # Layer 1 (input layer)
    tf.keras.layers.Dense(units=OUTPUT_SHAPE,
                          activation="softmax") # Layer 2 (output layer)
  ])

  # Compile the model
  model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(), # Our model wants to reduce this (how wrong its guesses are)
      optimizer=tf.keras.optimizers.Adam(), # A friend telling our model how to improve its guesses
      metrics=["accuracy"] # We'd like this to go up
  )

  # Build the model
  model.build(INPUT_SHAPE) # Let the model know what kind of inputs it'll be getting

  return model

In [ ]:
model = create_model()
model.summary()

In [ ]:
outputs = np.ones(shape=(1,1,1280))
outputs

## Creating Callbacks

Callbacks are helper functions a Model can use during training to do such things as save its progress, check its progress.

We'll create two callbacks.

1. One for Tensorboard which helps TRACK our Models Progress and

2. One for Early Stopping which prevents our model from training for too long.

### TENSORBOARD Callback

To setup a TensorBoard callback, we need to do 3 things:

1. Load the TensorBoard Extension

2. Create a TensorBoard Callback which is able to save logs to a Directory and pass it to our model's fit() function

3. Visualise our Models with the %TensorBoard magic function.

In [ ]:
# Load TensorBoard Notebook Extension
%load_ext tensorboard


In [ ]:
import datetime
# Create a function to build a TensorBoard Callback
def create_tensorboard_callback():
  # Create a log direcotry for storing Tensorboard Logs
  logdir = os.path.join("drive/MyDrive/Dog Vision/logs",
                        # Make it so the Logs gets tracked
                        datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  return tf.keras.callbacks.TensorBoard(logdir)

### Early Stopping Callback

Early Stopping helps stop our model by Overfitting by stopping training if a certain metric stops improving.

In [ ]:
# Create early stopping callback

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                  patience=3)

## Training a Model

Our first model is only going to train 1000 images, to make sure everything is working.

In [ ]:
NUM_EPOCHS = 100 #@param {type: "slider", min:10,max:100,step: 10}



In [ ]:
print("GPU","available" if tf.config.list_physical_devices("GPU") else "not available")

Let's create a function which trains a Model.

* Creating a Model using `create_model()`

* Setup a TensorBoard callback using `create_tensorboard_callback()`

* Call the `fit()` on our model passing it the Training Data, Validation Data, Number of epochs to train (NUM_EPOCHS) and the callbacks we'd like to use.

* Return the Model



In [ ]:
# Build a Function to train and return a Trained Model

def train_model():

  model = create_model()

  tensorboard = create_tensorboard_callback()

  # Fit the model to the data passing it the Callbacks we created
  model.fit(x=train_data,
            epochs=NUM_EPOCHS,
            validation_data=val_data,
            validation_freq = 1,
            callbacks = [tensorboard, early_stopping]
            )
  return model

In [ ]:
model = train_model()

**Question** It looks like our Model is OVERFITTING because it's performing far better on Training Set than Val Set
What are some ways to model Overfitting in Deep Learning Neural Networks?

**Note:** Overfitting to begin with is a good thing!

### Checking the TensorBoard Logs

The Tensorboard Magic Function (%tensorboard) will access the logs directly and visualise its contents.

In [ ]:
%tensorboard --logdir drive/MyDrive/Dog\ Vision/logs

## Making and Evaluating Predictions using a Trained Model

In [ ]:
val_data

In [ ]:
# Make Predictions on the Validation Data
predictions = model.predict(val_data, verbose=1)
predictions

In [ ]:
index= 42
print(predictions[index])
print(f"Max Value (Probability of Prediction): {np.max(predictions[index])}")
print(f"Sum: {np.sum(predictions[index])}")
print(f"Max Index: {np.argmax(predictions[index])}")
print(f"Predicted Label: {unique_breeds[np.argmax(predictions[index])]}")

Having the above functionality is great, but we want to do it at scale.

And it would be better to see the image the prediction is being made on..

**Note:** Prediction Probabilities are also known as Confidence Levels.

In [ ]:
# Turn Prediction Probabilities into their Respective Label (easier to understand)

def get_pred_label(prediction_probabilities):
  """
  Turns an Array of Prediction Probabilities into a Label
  """

  return unique_breeds[np.argmax(prediction_probabilities)]

# Get a Predicted Label based on an array of Prediciton Probabilities

pred_label = get_pred_label(predictions[81])

pred_label

In [ ]:
val_data

Now since our val_data is still in a BATCH DATASET, we'll have to UNBATCHIFY it to make Predictions on the Validation Images and then Compare those Predictions to the Validation Labels.

In [ ]:
images_ = []
labels_ = []

# Loop through Unbatched Data

for image, label in val_data.unbatch().as_numpy_iterator():
  images_.append(image)
  labels_.append(label)

images_[0] , labels_[0]

In [ ]:
get_pred_label(labels_[0])

In [ ]:
get_pred_label(predictions[0])

In [ ]:
def unbatchify(data):
  """
  Takes a Batched Dataset of (image,label) tensors and returns seperate arrays of images and labels.
  """
  images = []
  labels=[]

  for image, label in data.unbatch().as_numpy_iterator():
    images.append(image)
    labels.append(unique_breeds[np.argmax(label)])

  return images, labels

val_images, val_labels = unbatchify(val_data)
val_images[0] , val_labels[0]

Now we've got ways to get :

* Prediction Labels
* Validation Labels
* Validation Images

We'll create a Function which
* Takes an Array of Prediciton Probs, an Array of Truth Labels, and images and Integers.
* Convert the  Preds Probs to a predicted Label.

* Plot the Predicted Label, its predicted Probability, the Truth label and the Target Image on a Single Label.

In [ ]:
def plot_pred(prediction_probabilities, labels, images, n=1):
  """
  View the Prediction, Ground Truth, and Image for a sample n.
  """
  pred_prob, true_label, image = prediction_probabilities[n], labels[n], images[n]

  pred_label = get_pred_label(pred_prob)

  # Plot Image and Remove Ticks
  plt.imshow(image)
  plt.xticks([])
  plt.yticks([])

  # Change the Colour of the Title
  if pred_label == true_label:
    color = "green"
  else:
    color = "red"

  # Change the Plot Title to be Predicted
  plt.title("{} {:2.0f}% {}".format(pred_label, np.max(pred_prob) * 100,true_label), color= color)


In [ ]:
plot_pred(predictions,val_labels, val_images,n=2)

Now we've got one function to Visualise our Model's Predictions, let's make another to view top 10 PREDICTIONS.

This function will :
* Take an Input of Prediction Probabilities array and a Ground Truth Array and an Integer
* Find the Prediction using `get_pred_label()`
* Find the top 10:
  * Prediction Probabilites Indexes
  * Prediction Probabilities Values
  * Prediction Labels

In [ ]:
def plot_pred_conf(prediction_probabilities, labels, n=1):
  """
  Plots the top 10 Highest Prediction Confidences along with the Truth  Label for Sample n.
  """
  pred_prob, true_label = prediction_probabilities[n], labels[n]

  pred_label = get_pred_label(pred_prob)

  # Find the Top 10 Prediction
  top_10_pred_indexes = pred_prob.argsort()[-10:][::-1]

  # Find the top 10 Prediction Confidence Levels
  top_10_pred_values = pred_prob[top_10_pred_indexes]

  # Find the top 10 Prediciton Labels
  top_10_pred_labels = unique_breeds[top_10_pred_indexes]

  # Setup Plot
  top_plot = plt.bar(np.arange(len(top_10_pred_labels),),
                     top_10_pred_values,
                     color="grey")
  plt.xticks(np.arange(len(top_10_pred_labels)),labels=top_10_pred_labels,rotation="vertical")

  if np.isin(true_label, top_10_pred_labels):
    top_plot[np.argmax(top_10_pred_labels == true_label)].set_color("green")
  else:
    pass


In [ ]:
plot_pred_conf(predictions,val_labels, n=9)

Now we've got some functions to help us Visualise our Predictions and Evaluate our Model

In [ ]:
# Let's check out a Few Predictions and their Different Values
i_multiplier = 10
num_rows= 3
num_cols =2

num_images = num_rows * num_cols

plt.figure(figsize=(10*num_cols, 5 * num_cols))
for i in range(num_images):
  plt.subplot(num_rows, 2* num_cols, 2*i + 1)
  plot_pred(predictions,val_labels, val_images, n=i+i_multiplier)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_pred_conf(predictions, val_labels, i+i_multiplier)

plt.show()

### Saving our Model

In [ ]:
# Create a Function to save a model
def save_model(model,suffix=None):
  """
  Saves a Given Model in a Model's Directory and appends a Suffix
  """
  # Create a Model Directory pathname with current Time
  modeldir = os.path.join("drive/MyDrive/Dog Vision/models", datetime.datetime.now().strftime("%Y%m%d-%H%M%s"))
  model_path = modeldir + "-" + suffix + ".h5" #Save Format of Model
  print(f"Saving Model to {model_path}...")
  model.save(model_path)

  return model_path

In [ ]:
# Create a Function to Load a Model
def load_model(model_path):
  """
  Loads a Saved Model from a Specified Model
  """
  print(f"Loading Saved Model from: {model_path}")
  model = tf.keras.models.load_model(model_path,
                                     custom_objects={"KerasLayer":hub.KerasLayer})
  return model

In [ ]:
# Save our Model Trained on 1000 Images
save_model(model, suffix="1000-images-mobilenetv2-Adam")

In [ ]:
# Load a Trained Model
loaded_1000_image_model = load_model("/content/drive/MyDrive/Dog Vision/models/20241105-13361730813789-1000-images-mobilenetv2-Adam.h5")

In [ ]:
# Evaluate the Pre-saved Model
model.evaluate(val_data)

In [ ]:
# Evaluate the Loaded Model
loaded_1000_image_model.evaluate(val_data)

## Training a Big Dog Model (On the Full Data)

In [ ]:
# Create a Data Batch with the Full Dataset
full_data = create_data_batches(X,y)


In [ ]:
full_model = create_model()

In [ ]:
full_model_tensorboard = create_tensorboard_callback()

full_model_early_stopping = tf.keras.callbacks.EarlyStopping('accuracy',patience=3)



In [ ]:
full_model.fit(x=full_data, epochs=NUM_EPOCHS, callbacks=[full_model_tensorboard, full_model_early_stopping])

In [ ]:
loaded_full_model = load_model('/content/drive/MyDrive/Dog Vision/models/20241106-05251730870742-1000-images-mobilenetv2-Adam.h5')

## Making Predictions on the Test Dataset

Since our model has been trained on images in the form of Tensor batches, to make predictions on the Test Data, we'll have to get it into the same dataset.

Luckily we created `create_data_batches()` earlier which can take a list of filenames as input and convert them into Tensor Batches.

To make Predictions on the Test Data, we'll:
* Get the Test image filenames
* Convert the filenmaes into test data batches using `create_data_batches` and setting the `test_data` parameter to `True` (since the test data doesn't have labels)

* Make a Predicitons array by passign the test batches

In [ ]:
# Load test image filenames
test_path = "/content/drive/MyDrive/Dog Vision/test"
test_filenames = [test_path + "/" + fname for fname in os.listdir(test_path)]
test_filenames[:10]

In [ ]:
len(test_filenames)

In [ ]:
test_data = create_data_batches(test_filenames, test_data=True)

In [ ]:
test_data

In [ ]:
# Make Predictions on the Test Data Batch using the fully loaded Model
test_predictions = loaded_full_model.predict(test_data, verbose=1)

In [ ]:
# Save Predictions (NumPy Array) to csv file
np.savetxt("/content/drive/MyDrive/Dog Vision/preds_array.csv", test_predictions, delimiter=",")



In [ ]:
test_predictions = np.loadtxt("/content/drive/MyDrive/Dog Vision/preds_array.csv", delimiter=",")

## Preparing Dataset Predictions for Kaggle
www.kaggle.com/competitions/dog-breed-identification/overview/evaluation


In [ ]:
# Create a Pandas DataFrame with Empty columns
pred_df = pd.DataFrame(columns=["id"] + list(unique_breeds))
pred_df

In [ ]:
# Append Test Image ID's to Predictions DataFrame
test_ids = [os.path.splitext(path)[0] for path in os.listdir(test_path)]
pred_df["id"] = test_ids

In [ ]:
# Add the Prediction Probabilities to Each dog breed Column

pred_df[list(unique_breeds)] = test_predictions
pred_df.head()

In [ ]:
pred_df.to_csv("drive/MyDrive/Dog Vision/full_model_predictions_submission_1_mobilenetv2.csv",index=False)


## Making Predictions on Custom Images

To make Predictions on Custom Images

In [ ]:
# Get Custom Image File Path
import os
custom_path = "drive/MyDrive/Dog Vision/custom_images"
custom_image_paths = [custom_path + "/" + fname for fname in os.listdir(custom_path)]
custom_image_paths


In [ ]:
# Turn Customn Images into Batch Dataset
custom_data = create_data_batches(custom_image_paths)
custom_data

In [ ]:
custom_preds = loaded_full_model.predict(custom_data)
custom_preds

In [ ]:
custom_pred_labels = [get_pred_labels(custom_preds[i]) for i in range(len(custom_preds))]
custom_pred_labels

In [ ]:
# Get Custom Images (Our Unbatchify() won't work since we don't have the Labels)

custom_images = []

for image in custom_data.unbatch().as_numpy_iterator():
  custom_images.append(image)


In [ ]:
# Check Custom Image Predictions
plt.figure(figsize=(10,10))
for i, image in enumerate(custom_images):
  plt.subplot(1,3,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.title(custom_pred_labels[i])
  plt.imshow(image)

plt.show()
